In [ ]:
## Codes of fig5h-i
### merge single-cell bin
import os, sys
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import BisectingKMeans
from scipy.sparse import csr_matrix

def removeBiasGenes(adata):
    malat1 = adata.var_names.str.startswith('MALAT1')
    MTgenes = adata.var_names.str.startswith('MT')
    hb_genes = adata.var_names.str.contains('^HB[^(P)]')
    RPgenes = adata.var_names.str.startswith('RP') & adata.var_names.str.contains('-')
    RPgenes2 = adata.var_names.str.contains('^RP[SL]')
    CTCgenes = adata.var_names.str.startswith('CTC') & adata.var_names.str.contains('-')
    MIRgenes = adata.var_names.str.startswith('MIR')
    ACgenes = adata.var_names.str.contains('^AC[0-9]') & adata.var_names.str.contains('.')
    CTgenes = adata.var_names.str.startswith('CT') & adata.var_names.str.contains('-')
    LINCgenes = adata.var_names.str.contains('^LINC[0-9]')
    ALgenes = adata.var_names.str.contains('^AL') & adata.var_names.str.contains('.')

    remove_genes = malat1 | MTgenes | hb_genes | RPgenes | RPgenes2 | CTCgenes | MIRgenes | ACgenes | CTgenes | LINCgenes | ALgenes
    keep = np.invert(remove_genes)
    res = adata[:,keep]
    return res

def merge_big_cell(adata, resolution, n=30):
    adata_list = []
    for category in adata.obs[resolution].unique():
        sub_adata = adata[adata.obs[resolution] == category].copy()
        adata_list.append(sub_adata)
    merged_adatas = []
    merged_cluster_labels = pd.DataFrame(columns=["cell_id", "merged_cluster"])
    for i, st_adata in enumerate(adata_list):
        t = st_adata.shape[0] // n
        if t == 0:
            t = 1
        X = st_adata.obs[["x", "y"]].values
        spectral_clustering = BisectingKMeans(n_clusters=t, bisecting_strategy="largest_cluster", random_state=0)
        cluster_labels = spectral_clustering.fit_predict(X)
        # cluster_sizes = np.bincount(cluster_labels)
        # sorted_clusters = np.argsort(cluster_sizes)[::-1]
        # max_cluster_index = sorted_clusters[0]
        # while max_cluster_index
        cluster = st_adata.obs[resolution].iloc[0]
        adata_idx = adata.obs[resolution].unique().to_list().index(cluster)
        merged_cluster_labels_i = pd.DataFrame(
            list(zip(st_adata.obs_names, ["%s_%s" % (i - 1, adata_idx) for i in cluster_labels])),
            columns=["cell_id", "merged_cluster"],
        )
        merged_cluster_labels = pd.concat([merged_cluster_labels, merged_cluster_labels_i], axis=0)
        # Calculate the center coordinates and gene count of each cluster
        cluster_centers = []
        cluster_gene_counts = []
        cluster_cell_counts = []
        for cluster_id in np.unique(cluster_labels):
            cluster_spots = st_adata.obs.index[cluster_labels == cluster_id]
            cluster_center = np.mean(st_adata.obs.loc[cluster_spots, ["x", "y"]], axis=0)
            cluster_gene_count = np.sum(st_adata[cluster_spots].X, axis=0)
            cluster_centers.append(cluster_center)
            cluster_gene_counts.append(cluster_gene_count)
            cluster_cell_counts.append(cluster_spots.shape[0])
        # build the merged adata object
        adata_cluster = sc.AnnData(
            X=np.array(cluster_gene_counts)[:, 0, :],
            obsm={"spatial": np.array(cluster_centers)},
            obs=pd.DataFrame(
                cluster_cell_counts, index=np.arange(len(cluster_centers)), columns=["cluster_cell_counts"]
            ),
            var=adata.var,
        )
        adata_cluster.obs["merged_cluster"] = st_adata.obs[resolution].iloc[0]
        merged_adatas.append(adata_cluster)
    merged_cluster_labels.to_csv(f"merged_cluster_labels.txt", sep="\t", index=None)
    # merge adata object
    merged_adata = sc.AnnData.concatenate(*merged_adatas, join="outer")
    return merged_adata


adata = sc.read_h5ad('anno_rawdata.h5ad')
adata = adata[adata.obs['annotated_cluster'].isin(['IBC cells','DCIS cells'])].copy()

merged_adata = merge_big_cell(adata, 'annotated_cluster', 30)
sparse_X = csr_matrix(merged_adata.X)
merged_adata.X = sparse_X
merged_adata.obs['x'] = merged_adata.obsm['spatial'][:, 0]
merged_adata.obs['y'] = merged_adata.obsm['spatial'][:, 1]
merged_adata = removeBiasGenes(merged_adata)
merged_adata.write_h5ad('tumor_bigcell.h5ad')


In [ ]:
#### this the the function to generate the ranked gene list between different samples sets
# samplesCluster: samples in each cluster
# mat: the expression matrix, row is genes, col is samples

## DEG_wilcox calculate the DEG between different clusters, give the list including the 
## samples in each cluster, and the expression matrix, return the matrix of fc, pvalue
library(ggplot2)
library(ggalluvial)
library(svglite)
library(Seurat)
library(SeuratData)
library(MuDataSeurat)
# options(stringsAsFactors = FALSE, future.globals.maxSize=20000*1024^2)

DEG_wilcox <- function(samplesCluster, mat){
  library(future.apply)
  
  DEG <- list()
  if(length(samplesCluster) < 3){
    n <- 1
  }else{
    n <- length(samplesCluster)
  }
  
  options(future.globals.maxSize= 8912896000)
  plan(multisession, workers=20)
  
  for(i in 1:n){
    selectSamples <- samplesCluster[[i]]
    otherSamples <- colnames(mat)[!(colnames(mat)%in%selectSamples)]
    
    DEG[[i]] <- matrix(0, nrow = nrow(mat), ncol = 6)
    rownames(DEG[[i]]) <- rownames(mat)
    colnames(DEG[[i]]) <- c("Stat", "Pval", "MeanSelect", "MeanOther", "FC", "FDR")
    
    wiltest <- future_lapply(seq(nrow(mat)), function(x){
      wilcox.test(mat[x, selectSamples], mat[x, otherSamples], alternative = "two.sided")
    })
    
    DEG[[i]][,1] <- unlist(lapply(wiltest, function(x) x$statistic))
    DEG[[i]][,2] <- unlist(lapply(wiltest, function(x) x$p.value))
    DEG[[i]][,3] <- rowMeans(mat[, selectSamples])
    DEG[[i]][,4] <- rowMeans(mat[, otherSamples])
    DEG[[i]][,5] <- log2(DEG[[i]][,3]/DEG[[i]][,4])
    
    # for(j in 1:nrow(mat)){
    #   #tmp <- wilcox.test(mat[j, selectSamples], mat[j, otherSamples], alternative = "two.sided")
    # 
    #   DEG[[i]][j,1] <- wiltest[[j]]$statistic
    #   DEG[[i]][j,2] <- wiltest[[j]]$p.value
    #   DEG[[i]][j,3] <- mean(mat[j, selectSamples])
    #   DEG[[i]][j,4] <- mean(mat[j, otherSamples])
    #   DEG[[i]][j,5] <- log2(mean(mat[j, selectSamples])/mean(mat[j, otherSamples]))
    # 
    #   rm(tmp)
    # }
    
    DEG[[i]][,6] <- p.adjust(p = DEG[[i]][,"Pval"], method = "fdr")
    DEG[[i]] <- DEG[[i]][order(DEG[[i]][,5], decreasing = TRUE),]
    
    rm(selectSamples, otherSamples)
  }
  
  if(length(samplesCluster) < 3){
    names(DEG) <- names(samplesCluster)[1]
  }else{
    names(DEG) <- names(samplesCluster)
  }
  
  return(DEG)
}


od <- 'DEG_result/'
setwd(od)

sc <- ReadH5AD('tumor_bigcell.h5ad')
sc <- NormalizeData(sc, normalization.method = "LogNormalize", scale.factor = 10000)

samplesCluster = list()
for (subclo in sort(unique(sc$merged_cluster))) {
  samplesCluster[[subclo]] = rownames(sc@meta.data)[sc@meta.data$merged_cluster==subclo]
}
#options(future.globals.maxSize= 8912896000)
degs_raw = DEG_wilcox2(samplesCluster, as.matrix(sc@assays$RNA@data))
degs_raw = data.frame(degs_raw)
degs_raw$gene <- rownames(degs_raw)
degs_raw <- dplyr::filter(degs_raw, !grepl('MT', gene))
degs_raw <- dplyr::filter(degs_raw, !grepl('RP', gene))
write.csv(degs_raw, 'deg.csv')